In [11]:
from pathlib import Path
from infrastructure.robot.fleet import get_fleet_data
from wayve.core.calibration.load_calibration import load_calibration_textproto, load_v2_textproto
from wayve.core.calibration.backcompat_support import convert_to_v2

REPO_ROOT = "/home/benjin/Development/wcs/WayveCode"
CALIBRATION_DIR = Path(REPO_ROOT) / "infrastructure" / "robot" / "config"

VEHICLES = [entry.name for entry in CALIBRATION_DIR.iterdir() if entry.is_dir()]


In [12]:
def get_calibration_gen1(vehicle_dir: Path, prior: bool):
    if prior:
        textproto_file = vehicle_dir / "bb" / "vehicle_calibration_default.textproto"

    else:
        textproto_file = vehicle_dir / "bb" / "vehicle_calibration.textproto"

    calib = load_calibration_textproto(textproto_file)
    calib = convert_to_v2(calib)
    
    return calib
    

def get_calibration_gen2(vehicle_dir: Path, prior: bool):
    if prior:
        textproto_file = vehicle_dir / "wayve-interfaces-protobuf-VehicleCalibration_default.textproto"
    
    else:
        textproto_file = vehicle_dir / "wayve-interfaces-protobuf-VehicleCalibration_.textproto"

    calib = load_v2_textproto(textproto_file)
    return calib

def is_gen1(vehicle_dir: Path) -> bool:
    calib_exists = (vehicle_dir / "bb" / "vehicle_calibration.textproto").exists()
    prior_calib_exists = (vehicle_dir / "bb" / "vehicle_calibration_default.textproto").exists()
    return calib_exists and prior_calib_exists

def is_gen2(vehicle_dir: Path) -> bool:
    calib_exists = (vehicle_dir / "wayve-interfaces-protobuf-VehicleCalibration_.textproto").exists()
    prior_calib_exists = (vehicle_dir / "wayve-interfaces-protobuf-VehicleCalibration_default.textproto").exists()
    return calib_exists and prior_calib_exists


def get_calibration(vehicle, prior=False):
    vehicle_dir = CALIBRATION_DIR / vehicle

    if is_gen1(vehicle_dir):
        calib = get_calibration_gen1(vehicle_dir, prior)
    elif is_gen2(vehicle_dir):
        calib = get_calibration_gen2(vehicle_dir, prior)
    else:
        print(f"Vehicle directory does not seem to be either gen1 or gen2 for {vehicle}")
        return None

    return calib

def assert_equality_serial_numbers(calib, calib_prior):
    # Check that the same sensor calibration entries exist
    calib_sensors = set(calib.sensor_calibrations.keys())
    calib_prior_sensors = set(calib_prior.sensor_calibrations.keys())
    if calib_sensors != calib_prior_sensors:
        return f"Sensor calibration entries do not match calib {calib_sensors} vs calib_prior {calib_prior_sensors}"
    
    err_msgs = []

    for sensor in calib_sensors:
        calib_sensor_serial = calib.sensor_calibrations[sensor].metadata.serial_number
        calib_prior_sensor_serial = calib_prior.sensor_calibrations[sensor].metadata.serial_number

        if calib_sensor_serial != calib_prior_sensor_serial:
            err_msgs.append(f"Serial number mismatch for sensor {sensor}: {calib_sensor_serial} vs {calib_prior_sensor_serial}")
        
    if len(err_msgs) > 0:
        return "\n" + "\n".join(err_msgs)
    
    return None

def assert_equality_camera_intrinsics(calib, calib_prior):
    # Check that the same sensor calibration entries exist
    calib_sensors = set(calib.sensor_calibrations.keys())
    calib_prior_sensors = set(calib_prior.sensor_calibrations.keys())
    if calib_sensors != calib_prior_sensors:
        return f"Sensor calibration entries do not match calib {calib_sensors} vs calib_prior {calib_prior_sensors}"
    
    err_msgs = []

    for sensor in calib_sensors:
        calib_sensor_intrinsics = calib.sensor_calibrations[sensor].camera_intrinsics
        calib_prior_sensor_intrinsics = calib_prior.sensor_calibrations[sensor].camera_intrinsics

        if calib_sensor_intrinsics.camera_geometry != calib_prior_sensor_intrinsics.camera_geometry:
            err_msgs.append(f"Camera geometry mismatch for sensor {sensor}:\n{calib_sensor_intrinsics.camera_geometry}vs\n{calib_prior_sensor_intrinsics.camera_geometry}")

        if calib_sensor_intrinsics.focal_length_px != calib_prior_sensor_intrinsics.focal_length_px:
            err_msgs.append(f"Focal length mismatch for sensor {sensor}:\n{calib_sensor_intrinsics.focal_length_px}vs\n{calib_prior_sensor_intrinsics.focal_length_px}")
        
        if calib_sensor_intrinsics.principal_point_px != calib_prior_sensor_intrinsics.principal_point_px:
            err_msgs.append(f"Principal point mismatch for sensor {sensor}:\n{calib_sensor_intrinsics.principal_point_px}vs\n{calib_prior_sensor_intrinsics.principal_point_px}")

        if calib_sensor_intrinsics.dist_coeffs[:4] != calib_prior_sensor_intrinsics.dist_coeffs[:4]:
            err_msgs.append(f"Distortion coefficients mismatch for sensor {sensor}:\n{calib_sensor_intrinsics.dist_coeffs[:4]}\nvs\n{calib_prior_sensor_intrinsics.dist_coeffs[:4]}")

    if len(err_msgs) > 0:
        return "\n" + "\n".join(err_msgs)
    
    return None

problematic_vehicles = []

for vehicle in VEHICLES:

    print(f"////////////////////////////////{vehicle}////////////////////////////////")

    calib = get_calibration(vehicle, prior=False)
    calib_prior = get_calibration(vehicle, prior=True)

    if calib is None or calib_prior is None:
        continue

    err_sn = assert_equality_serial_numbers(calib, calib_prior)
    if err_sn is not None:
        print(f"In assert_equality_serial_numbers for {vehicle} got error: {err_sn}")

    err_intr = assert_equality_camera_intrinsics(calib, calib_prior)
    if err_intr is not None:
        print(f"In assert_equality_camera_intrinsics for {vehicle} got error: {err_intr}")
    
    if err_sn is not None or err_intr is not None:
        problematic_vehicles.append(vehicle)


////////////////////////////////avon////////////////////////////////
In assert_equality_camera_intrinsics for avon got error: 
Focal length mismatch for sensor back-left-leftward:
x: 1145.6
y: 1143.8
vs
x: 1144.41748
y: 1142.89929

Principal point mismatch for sensor back-left-leftward:
x: 1010.9
y: 640.23
vs
x: 1015.80499
y: 645.220703

Distortion coefficients mismatch for sensor back-left-leftward:
[-0.019493, -0.055636, 0.049564, -0.018839]
vs
[-0.01721392, -0.05743259, 0.04944835, -0.01848868]
Focal length mismatch for sensor back-right-rightward:
x: 1147.9
y: 1146.3
vs
x: 1144.41748
y: 1142.89929

Principal point mismatch for sensor back-right-rightward:
x: 1009.8
y: 641.11
vs
x: 1015.80499
y: 645.220703

Distortion coefficients mismatch for sensor back-right-rightward:
[-0.013371, -0.065532, 0.054222, -0.019]
vs
[-0.01721392, -0.05743259, 0.04944835, -0.01848868]
Focal length mismatch for sensor front-left-leftward:
x: 1140.7
y: 1138.9
vs
x: 1144.41748
y: 1142.89929

Principal po

In [13]:
problematic_vehicles

['avon',
 'beni',
 'brizo',
 'congo',
 'ebisu',
 'ganges',
 'hudson',
 'nammu',
 'neptune',
 'nereus',
 'oya',
 'po',
 'poseidon',
 'sedna',
 'shire',
 'thames',
 'mekong',
 'lbn00003']